In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
from datetime import datetime
import datetime
import pandas_datareader.data as web

In [2]:
start = datetime.datetime(2019, 9, 27)
end = datetime.datetime(2020, 10, 7)
btc = web.DataReader('BTC-USD','yahoo', start, end)

In [3]:
btc = btc[~btc.index.duplicated(keep='first')]

In [4]:
btc = btc.drop(columns=['High','Low','Open','Close','Volume'])

In [5]:
btc.columns = ['Price']

In [6]:
btc = btc.reset_index()

In [7]:
r = pd.date_range(start=btc.Date.min(), end=btc.Date.max())
btc = btc.set_index('Date').reindex(r).rename_axis('Date').reset_index()

In [8]:
btc['Date'] = pd.to_datetime(btc['Date'])
btc.set_index('Date',inplace=True)

In [9]:
btc = btc.fillna(method='ffill')
btc = btc.fillna(method='bfill')

In [10]:
btc = btc.loc['2019-09-27':'2020-10-07']

In [11]:
start = datetime.datetime(2019, 9, 27)
end = datetime.datetime(2020, 10, 7)
sc = web.DataReader('SC-USD','yahoo', start, end)

In [12]:
sc = sc.loc['2019-09-27':'2020-10-07']

In [13]:
sc = sc[~sc.index.duplicated(keep='first')]

In [14]:
sc = sc.drop(columns=['High','Low','Open','Close'])

In [15]:
sc.columns = ['Price','Volume']

In [16]:
sc = sc.reset_index()

In [17]:
r = pd.date_range(start=sc.Date.min(), end=sc.Date.max())
sc = sc.set_index('Date').reindex(r).rename_axis('Date').reset_index()

In [18]:
sc['Date'] = pd.to_datetime(sc['Date'])
sc.set_index('Date',inplace=True)

In [19]:
sc = sc.fillna(method='ffill')
sc = sc.fillna(method='bfill')

In [20]:
import requests
from pandas.io.json import json_normalize

In [21]:
url = 'https://siastats.info/dbs/miningdb.json'
sc_hr = pd.read_json(url)

In [22]:
sc_hr['date'] = pd.date_range(start='6/6/2015', periods=len(sc_hr), freq='D')

In [23]:
sc_hr = sc_hr.drop(columns=['time','difficulty','blocktime'])

In [24]:
sc_hr['date'] = pd.to_datetime(sc_hr['date'])
sc_hr.set_index('date',inplace=True)

In [25]:
sc_hr = sc_hr.loc['2019-09-27':'2020-10-07']

In [26]:
url1 = 'https://siastats.info/dbs/activecontracts.json'
sc_contr = pd.read_json(url1)

In [27]:
sc_contr['date'] = pd.date_range(start='6/6/2015', periods=len(sc_contr), freq='D')

In [28]:
sc_contr = sc_contr.drop(columns=['time','activecontractsize','activecontractcost'])

In [29]:
sc_contr['date'] = pd.to_datetime(sc_contr['date'])
sc_contr.set_index('date',inplace=True)

In [30]:
sc_contr = sc_contr.loc['2019-09-27':'2020-10-07']

In [31]:
url2 = 'https://siastats.info/dbs/storage.json'
sc_stor = pd.read_json(url2)

In [32]:
sc_stor['date'] = pd.date_range(start='9/11/2017', periods=len(sc_stor), freq='D')

In [33]:
sc_stor = sc_stor.drop(columns=['total'])

In [34]:
sc_stor['date'] = pd.to_datetime(sc_stor['date'])
sc_stor.set_index('date',inplace=True)

In [35]:
sc_stor = sc_stor.loc['2019-09-27':'2020-10-07']

In [36]:
url3 = 'https://siastats.info/dbs/transactionsdb.json'
sc_tx = pd.read_json(url3)

In [37]:
sc_tx['date'] = pd.date_range(start='6/6/2015', periods=len(sc_tx), freq='D')

In [38]:
sc_tx = sc_tx.drop(columns=['time','totaltx'])

In [39]:
sc_tx['date'] = pd.to_datetime(sc_tx['date'])
sc_tx.set_index('date',inplace=True)

In [40]:
sc_tx = sc_tx.loc['2019-09-27':'2020-10-07']

In [41]:
frames = [btc,sc,sc_hr,sc_contr,sc_stor,sc_tx]

In [42]:
sc_multy = pd.concat(frames,sort=False,axis=1)

In [43]:
sc_multy.columns = ['BTC_Price','SC_Volume','SC_Price','Hashrate','Contracts','Used_Storage','Daily_Tx']

In [44]:
sc_multy = sc_multy.fillna(method='ffill')
sc_multy = sc_multy.fillna(method='bfill')

In [45]:
sc_multy = sc_multy.reindex(columns = ['BTC_Price','SC_Volume','Hashrate','Contracts','Used_Storage','Daily_Tx','SC_Price'])

In [46]:
sc_multy.tail()

,BTC_Price,SC_Volume,Hashrate,Contracts,Used_Storage,Daily_Tx,SC_Price
2020-10-03,10684.428711,1011766.0,9993.634212,195861.0,706.84,11963.0,0.002756
2020-10-04,10804.000977,1624328.0,8426.050178,195124.0,706.84,14210.0,0.002826
2020-10-05,10621.664062,1354025.0,8338.793499,192975.0,706.84,19466.0,0.002637
2020-10-06,10621.664062,1354025.0,8632.116299,191107.0,706.84,31968.0,0.002637
2020-10-07,10626.291992,1487110.0,8632.116299,191107.0,706.84,31968.0,0.002652


In [75]:
sc_multy = sc_multy.drop(columns='BTC_Price')

In [76]:
x = sc_multy.iloc[:,:-1].values
y = sc_multy.iloc[:,-1].values

In [77]:
from sklearn.tree import DecisionTreeRegressor
regressor = DecisionTreeRegressor(random_state = 0)
regressor.fit(x, y)

DecisionTreeRegressor(ccp_alpha=0.0, criterion='mse', max_depth=None,
                      max_features=None, max_leaf_nodes=None,
                      min_impurity_decrease=0.0, min_impurity_split=None,
                      min_samples_leaf=1, min_samples_split=2,
                      min_weight_fraction_leaf=0.0, presort='deprecated',
                      random_state=0, splitter='best')

In [83]:
regressor.predict([[150000, 100000, 200000, 750, 30000]])

array([0.00296385])